In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import pandas as pd

In [2]:
n_instance = 10
n_agents = 2
actions_space = np.arange(1.2, 2.3, 0.05)
quality = np.ones(n_agents) * 2
margin_cost = np.ones(n_agents)
margin_cost[1] = 0.5
horizon = 1 / 4
a0 = 0
n_actions = actions_space.size
state_ravel = (n_actions,) * n_agents

In [19]:
actions_space

array([1.2 , 1.25, 1.3 , 1.35, 1.4 , 1.45, 1.5 , 1.55, 1.6 , 1.65, 1.7 ,
       1.75, 1.8 , 1.85, 1.9 , 1.95, 2.  , 2.05, 2.1 , 2.15, 2.2 , 2.25])

In [3]:
def replay_classic_reward(price):
    # Compute profits for all agents
#     price = actions_space[action]
    demand = np.exp((quality - price) / horizon)
    demand = demand / (np.sum(demand) + np.exp(a0 / horizon))
    reward = np.multiply(price - margin_cost, demand)
    return reward

In [4]:
with open('asyact_price.pickle', 'rb') as fp:
    a_state = pickle.load(fp)

with open('asyact_Q.pickle', 'rb') as fp:
    a_Q = pickle.load(fp)

In [5]:
with open('asyreward_Q.pickle', 'rb') as fp:
    r_Q = pickle.load(fp)

with open('asyreward_price.pickle', 'rb') as fp:
    r_state = pickle.load(fp)

In [ ]:
r_Q[0].shape

In [9]:
# Reward matrix calculator
count = 0
N = 100000
monopoly = replay_classic_reward(np.array([2.198, 1.698])).mean()
nash = replay_classic_reward(np.array([1.372, 1.204])).mean()

# ss = np.random.SeedSequence(12345)
# # Last a few rngs for sampling memories
# child_seeds = ss.spawn(n_agents)
# rng = [np.random.default_rng(s) for s in child_seeds]

ratio = np.zeros(n_instance)

# converged price
converged = np.zeros(n_agents)

for agent0 in range(n_instance):
    for agent1 in [agent0]:
        sim_Q = np.zeros((n_agents, n_actions**n_agents, n_actions))
        sim_Q[0, :, :] = r_Q[agent0][0, :, :]
        sim_Q[1, :, :] = r_Q[agent1][1, :, :]
        state = r_state[agent0][-1]
#         init = np.zeros(n_agents, dtype=int)
#         for i in range(n_agents):
#             init[i] = rng[i].integers(0, n_actions, size=1)
#         state = np.ravel_multi_index(init, state_ravel)
#         state = np.random.randint(0, n_actions**n_agents, size=1)
        action = np.zeros(n_agents, dtype=int)
        reward = np.zeros(n_agents)
        for k in range(N):
            # For each agent, select and perform an action
            for i in range(n_agents):
                action[i] = sim_Q[i, state].argmax()
            if k > N - 10001:
                converged += actions_space[action]
                if action[0] > action[1]:
                    count += 1
#                 print('Instance', agent0, 'Instance', agent1, action)
                reward += replay_classic_reward(actions_space[action])
#             # Move to the next state
            state = np.ravel_multi_index(action, state_ravel)
            avg = np.sum(reward)/10001/n_agents
            ratio[agent0] = (avg - nash)/(monopoly - nash)
        print('Instance', agent0, 'vs', agent1, avg, 'ratio', ratio[agent0])

Instance 0 vs 0 0.3564631708083744 ratio 0.3677252347145463
Instance 1 vs 1 0.42472570912413754 ratio 0.7343078212319304
Instance 2 vs 2 0.4059383283814131 ratio 0.6334160759719324
Instance 3 vs 3 0.3752130741660772 ratio 0.46841571941383076
Instance 4 vs 4 0.356753990886896 ratio 0.36928699287980077
Instance 5 vs 5 0.41313305028778613 ratio 0.6720530770384819
Instance 6 vs 6 0.3753375530735734 ratio 0.46908419440762233
Instance 7 vs 7 0.39120624398290804 ratio 0.5543020298663646
Instance 8 vs 8 0.3998168714132922 ratio 0.6005427081776025
Instance 9 vs 9 0.36627096786040814 ratio 0.4203949376010834


In [10]:
converged/10000/10

array([1.785285, 1.870372])

In [11]:
# action based ratio
a_ratio = np.array([0.60938436, 0.79874612, 0.61134698, 0.63048103, 0.61707901,
                    0.51815132, 0.73782273, 0.7805825 , 0.64445768, 0.63972649])
# count/10000/10 = 0.5334399999999999
# Average price array([1.8161735, 1.7340945])

In [12]:
# reward based profit ratio
r_ratio = np.array([0.36772523, 0.73427518, 0.63341608, 0.4683863 , 0.36928699,
                    0.67205308, 0.46897823, 0.55435253, 0.60065238, 0.42052928])
# count/10000/10 = 0.3325
# average price array([1.785285, 1.870372])

In [17]:
a_ratio.mean(), a_ratio.std()

(0.658777822, 0.08261254899344067)

In [18]:
r_ratio.mean(), r_ratio.std()

(0.528965528, 0.12240993874629502)

In [ ]:
# act_hist = np.zeros((n_instance, 20, n_agents), dtype=int)
# price_hist = np.zeros((n_instance, 20, n_agents))
# reward_hist = np.zeros((n_instance, 20, n_agents))

# count = 0
# for ins in range(n_instance):
#     for k in range(20):
#         act_hist[ins, k, :] = np.unravel_index(r_state[ins][k], state_ravel)
#         price_hist[ins, k, :] = actions_space[act_hist[ins, k, :]]
#         reward_hist[ins, k, :] = replay_classic_reward(price_hist[ins, k, :])
#         if price_hist[ins, k, 0] > price_hist[ins, k, 1]:
#             count += 1
# (reward_hist.mean(axis=(1,2)) - nash)/(monopoly - nash)